# pandas로 파일 불러오기
- read_csv()
- read_tables() : 기본 구분자가 \t 일뿐 read_csv와 차이가 없다. 왜 만들었나.
- 옵션들
    - path : 파일 위치와 파일이름
    - sep 또는 delimiter : 열을 구분하기 위해 사용할 문자
    - header : 열 이름(색인)으로 사용할 skip 이후 줄 번호(기본값 : 0), 헤더가 없으면 None index_col : 행 인덱스로 사용할 열 번호 또는 이름 또는 리스트
    - names : 열 이름으로 사용할 리스트(header=None일 때 사용)
    - na_values : NA 값으로 처리할 값
    - nrows : 파일을 몇 줄을 읽을 것인지 지정
    - encoding : 인코딩 종류를 지정 (기본값 “utf-8”)
    - thousands : 숫자의 천 단위 구분기호 지정 (예: thousands=“,”)

In [4]:
import pandas as pd

path = '/Users/jsha/gjai/python_basic/pytest_basic/'
df = pd.read_csv(path+'occupancy.csv', header=0, encoding='cp949')
df.head()

,id,date,Temperature,Humidity,Light,CO2,HumidityRatio,Occupancy
0,1,2015-02-04 17:51,23.18,27.2720,426.0,721.25,0.004793,1
1,2,2015-02-04 17:51,23.15,27.2675,429.5,714.00,0.004783,1
2,3,2015-02-04 17:53,23.15,27.2450,426.0,713.50,0.004779,1
3,4,2015-02-04 17:54,23.15,27.2000,426.0,708.25,0.004772,1
4,5,2015-02-04 17:55,23.10,27.2000,426.0,704.50,0.004757,1


# pandas로 csv 파일 저장하기

# 데이터 합치기

## DataFrame 끼리 결합

### merge() 기본 사용법
 - merge()는 기본적으로 옵션 key를 중심으로 합친다.

In [5]:
df1 = pd.DataFrame({"key":["a", "b", "c"], "value1":[100, 200, 300]})
print(df1)

df2 = pd.DataFrame({"key":["c", "b", "a"], "value2":[500, 300, 400]})
print(df2)

df3 = pd.merge(df1, df2, on="key")
print(df3)

  key  value1
0   a     100
1   b     200
2   c     300
  key  value2
0   c     500
1   b     300
2   a     400
  key  value1  value2
0   a     100     400
1   b     200     300
2   c     300     500


### 동일한 컬럼을 가진 경우1 - merge() 

In [6]:
df1 = pd.DataFrame({"key":["a", "b", "c"], "value":[100, 200, 300]})
print(df1)

df2 = pd.DataFrame({"key":["c", "b", "a"], "value":[500, 300, 400]})
print(df2)

df3 = pd.merge(df1, df2, on="key")
print(df3)

  key  value
0   a    100
1   b    200
2   c    300
  key  value
0   c    500
1   b    300
2   a    400
  key  value_x  value_y
0   a      100      400
1   b      200      300
2   c      300      500


### 동일한 컬럼을 가진 경우2 - merge(suffixes=)
- 같은 컬럼을 가진 결합에서 옵션 suffixes 사용

In [8]:
df4 = pd.merge(df1, df2, on="key", suffixes=("_df1", "_df2"))
print(df4)

  key  value_df1  value_df2
0   a        100        400
1   b        200        300
2   c        300        500


### 동일한 컬럼을 가진 경우3 - concat()
- 데이터프레임이 같은 컬럼을 가지고 있고, 그대로 이어붙일 때는 pd.concat()을 사용한다
- **데이터를 리스트로 변환한 뒤 넣어야 한다.**
- 인덱스를 리셋하지 않으면 결합되면서 인덱스가 혼란스러워진다.
- 이럴 때, 옵션 ignore_index=True를 사용
- 참고로, reset_index()함수를 써도 되는데, 이건 기존 index를 새로운 컬럼에 집어넣어서 조금 귀찮다. 확인해 보자.


In [13]:
df5 = pd.concat([df1, df2])
print(df5)

df6 = df5.reset_index()
print(df6)

df7 = pd.concat([df1, df2], ignore_index=True)
print(df7)

  key  value
0   a    100
1   b    200
2   c    300
0   c    500
1   b    300
2   a    400
   index key  value
0      0   a    100
1      1   b    200
2      2   c    300
3      0   c    500
4      1   b    300
5      2   a    400
  key  value
0   a    100
1   b    200
2   c    300
3   c    500
4   b    300
5   a    400


## Series 끼리 결합 - concat()
- pd.concat()을 사용한다.
- 옵션 axis=0 or 1 을 이용하여 붙이는 방향을 바꿀 수 있다.

In [16]:
s1 = pd.Series([1, 2, 3])
s2 = pd.Series([5, 6, 7, 8])
s3 = pd.Series([9, 10, 11])
s4 = pd.concat([s1, s2, s3], ignore_index=True)
print(s4)
s5 = pd.concat([s1, s2, s3], ignore_index=True, axis=1)
print(s5)

0     1
1     2
2     3
3     5
4     6
5     7
6     8
7     9
8    10
9    11
dtype: int64
     0  1     2
0  1.0  5   9.0
1  2.0  6  10.0
2  3.0  7  11.0
3  NaN  8   NaN


## DataFrame과 Series의 결합
- DataFrame에 컬럼을 하나 더 만들면서 Series를 넣는 방식

In [19]:
data = {"mid":[80, 90, 85], "report":[10, 8, 9], "final":[85, 70, 90]}
df = pd.DataFrame(data)
print(df)
print("*"*30)
bonus = pd.Series([3, 5, 2])

df["bonus"] = bonus
print(df)

   mid  report  final
0   80      10     85
1   90       8     70
2   85       9     90
******************************
   mid  report  final  bonus
0   80      10     85      3
1   90       8     70      5
2   85       9     90      2


# 중복 제거

# 데이터 값 바꾸기

## map() - 사전형이나 함수형으로 데이터 치환

## apply() - 여러 컬럼을 함수형으로 데이터 치환